In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import torch.nn.functional as F
from datetime import datetime
from torchvision import datasets

In [2]:
class_number = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
RANDOM_SEED = 42
lr = 0.001

In [3]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root="./mnist_data", train=True, transform=transform, download=True)
valid_dataset = datasets.MNIST(root="./mnist_data", train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=32, shuffle=False)

In [16]:
class LeNet5(nn.Module):
    def __init__(self, class_number):
        super(LeNet5, self).__init__()   
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            ## (32, 6, h, w)
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2), # >> feat. map size goes half
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )
        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=class_number)
        )
        
    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1) 
        logits = self.classifier(x) # logits for each class
        probs = F.softmax(logits, dim=1) #probs for each class
        
        return logits, probs

In [5]:
def get_acc(model, data_loader, device) :
    
    correct_pred = 0
    n = 0
    
    model.eval()
    
    with torch.no_grad() :
        for x, y_true in data_loader:
            x = x.to(device)
            y = y_true.to(device)
            
            _, y_prob = model(x)
            _, predicted_labels = torch.max(y_prob, 1)
            
            
    return correct_pred.float() / n

In [6]:
def train(train_loader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0
    
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        
        y_hat, _ = model(x)
        loss = criterion(y_hat, y)
        running_loss += loss.item() * x.size(0)
        
        loss.backward()
        optimizer.step()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    
    return model, optimizer, epoch_loss

In [7]:
def validate(valid_loader, model, criterion, device):
    model.eval()
    running_loss = 0
    
    for x, y_true in valid_loader:
        x = x.to(device)
        y = y_true.to(device)
        
        y_hat, _ = model(x)
        loss = criterion(y_hat, y)
        running_loss += loss.item() * x.size(0)
        
    epoch_loss = running_loss / len(valid_loader.dataset)
            
            
    return model, epoch_loss

In [8]:
def training_loop(model, criterion, optimizer, train_loader, valid_loader,epochs,device,pring_every=1):
    
    train_losses = []
    valid_losses = []
    
    for epoch in range(0, epochs):
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer, device)
        train_losses.append(train_loss)
    
        with torch.no_grad():
            model, valid_loss = validate(valid_loader, model, criterion, device)
            valid_losses.append(valid_loss)
            
        if epoch % pring_every == (pring_every - 1) :
            train_acc = get_acc(model, train_loader, device=device)
            valid_acc = get_acc(model, valid_loader, device=device)
            
            print(f"{datetime.now().time().replace(microsecond=0)}--------"
                  f"Epoch: {epoch + 1}\t"
                  f"train loss : {train_loss:.4f}\t"
                  f"Valid loss : {valid_loss:.4f}\t"
                  f"Train ACC : {100 * train_acc:.2f}\t"
                  f"Valid ACC : {100 * valid_acc:.2f}")
            
    return model, optimizer, (train_losses, valid_losses)

In [17]:
torch.manual_seed(RANDOM_SEED)
model = LeNet5(class_number).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
model, optimizer, _ = training_loop(model, criterion, optimizer, train_loader, valid_loader, class_number, DEVICE)

AttributeError: 'int' object has no attribute 'float'